# Single cell fluorescence/holographic trajectory plotting

Designed to measure a series of time-dependent single cell fluorescent/holographic intensities and then plot them on comparative graphs


```
Author: 
- Nathan J. Day (nathan.day.16@ucl.ac.uk)
```


### STEPS
1. Measure the intenstities of the cell trajectories and save out as csv and pandas dataframe
2. Plot these trajectories 


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy import ndimage
from skimage import io
from sklearn import preprocessing
from matplotlib import animation

## Measure 

Read single cell mask information and obtain fluorescent and phase stats. If the data already exists in a .csv and just needs to be plotted then skip to the plotter section.

#### Raw data (image) location

The image data needs to be organised inside the ```root_dir``` as follows:

```
phase/
    0000_phase.tif
    0001_phase.tif
    ...
gfp/
    0000_gfp.tif
    0001_gfp.tif
    ...
rfp/
    0000_rfp.tif
    0001_rfp.tif
    ...
```
And the corresponding mask data needs to be organised inside the ```mask_version``` folder in the ```root_dir``` as follows:

```
singlecell_labels_gold/labels_cell1a
                            0000_mask.tif
                            0001_mask.tif
                            ...
singlecell_labels_gold/labels_cell2a
                            0000_mask.tif
                            0001_mask.tif
                            ...
singlecell_labels_gold/labels_cell...
```
where the ```singlecell_labels_gold = mask_type``` at the time of writing

In [ ]:
root_path = "C:/Users/Nathan/Documents/data/fucci/"

gfp_dir = os.path.join(root_path, "gfp")
rfp_dir = os.path.join(root_path, "rfp")
phase_dir = os.path.join(root_path, "phase")

gfp_list = [m for m in os.listdir(gfp_dir) if m.endswith('.tif')]
rfp_list = [m for m in os.listdir(rfp_dir) if m.endswith('.tif')]
phase_list = [m for m in os.listdir(phase_dir) if m.endswith('.tif')]

cell_ids = ["1a", "1b", "2a", "2b", "3a", "3b", "4", "5"]
mask_type = "gold" #gold standard or modelv4, modelv5 etc
 

In [ ]:
for cell in cell_ids:

    masks_dir_name = "labels_cell" + cell 
    masks_dir = os.path.normpath(os.path.join(root_path, "singlecell_labels_gold", masks_dir_name))
    mask_list = [m for m in os.listdir(masks_dir) if m.endswith('modified.tif')] # add something to include unmodified if modified doesnt exist?

    ### DIRECTORY CREATION ###
    stats_dir = os.path.join(root_path, 'stats') 
    try:
    # Create target Directory
        os.mkdir(stats_dir)
        print("Directory " , stats_dir ,  " Created ") 
    except FileExistsError:
        print("Directory " , stats_dir ,  " already exists")
    
    mask_type_stats_dir = os.path.join(stats_dir, mask_type) 
    try:
    # Create target Directory
        os.mkdir(mask_type_stats_dir)
        print("Directory " , mask_type_stats_dir ,  " Created ") 
    except FileExistsError:
        print("Directory " , mask_type_stats_dir ,  " already exists")    
            
    stat_file_dir = os.path.normpath(os.path.join(mask_type_stats_dir, "cell"+cell))
    try:
    # Create target Directory
        os.mkdir(stat_file_dir)
        print("Directory " , stat_file_dir ,  " Created ") 
    except FileExistsError:
        print("Directory " , stat_file_dir ,  " already exists")        
        #create target file
    stat_file = os.path.normpath(os.path.join(stat_file_dir, 'stats.csv'))
    
    df = pd.DataFrame([], columns=['GFP Intensity', 'RFP Intensity', 'Phase Intensity', 'Area', 'x','y']) #is this assignment of df correct? or does it overwrite "count_cells" above
   
    for i, mask_name in enumerate(mask_list):
        #useful for troubleshooting
        #print(i)
        #print(cell) 
        
        mask = io.imread(os.path.normpath(os.path.join(masks_dir, mask_name)))
        labelled, n_labels = ndimage.label(mask)
        
        gfp = io.imread(os.path.join(gfp_dir,gfp_list[i]))
        gfp_pix_value = ndimage.sum(gfp, labelled, 1) 
    
        rfp = io.imread(os.path.join(rfp_dir,rfp_list[i]))
        rfp_pix_value = ndimage.sum(rfp, labelled, 1)
        
        phase = io.imread(os.path.join(phase_dir,phase_list[i]))
        phase_pix_value = ndimage.sum(phase, labelled, 1)

        
        df.loc[i,'GFP Intensity'] = gfp_pix_value 
        df.loc[i,'RFP Intensity'] = rfp_pix_value
        df.loc[i,'Phase Intensity'] = phase_pix_value 
        df.loc[i,'Area'] = ndimage.sum(mask)
        #df.loc[i,'Max'] =  ndimage.maximum(phase, labelled)

        centroid = ndimage.measurements.center_of_mass(mask)
        y = centroid[0]
        x = centroid[1]
        df.loc[i,'x'] = format(x, '.2f')
        df.loc[i,'y'] = format(y, '.2f')
    
    df.to_csv(stat_file, index=False)  
    print(df)
print('FIN')    

### Load data
If the data already exists in a .csv and just needs to be plotted then skip to this part, **but make sure you uncomment then following cell!**

+ Number of cells
+ Number of frames/timepoints (length of trajectory)

##### At time of writing this the data organisation is as follows
Stats.csv:

|   |GFP Intensity|RFP Intensity|Phase Intensity|Area| x | y |
|---|-------------|-------------|---------------|----|---|---|
|1  |             |             |               |    |   |   |
|2  |             |             |               |    |   |   |
|...|             |             |               |    |   |   |
|t  |             |             |               |    |   |   |

And the directory organisation is as follows:

```
root_path/stats/mask_version/
                          cell1/stats.csv
                          cell2/stats.csv
                          ...
```
Where ```mask_version``` could be any of the following
```
mask_version = { /gold
                 /modelv4
                 /modelv5
                     ... } 
```

In [3]:
### Data location ###
root_path = "C:/Users/Nathan/data/fucci/stats/gold"
cell_ids = ["1a", "1b", "2a", "2b", "3a", "3b", "4", "5"]
stat_location = os.path.join(cell_id, "stats.csv")
norm_df = pd.read_csv(os.path.join(root_path, stat_location))


In [4]:
def colour_picker(variable_title):
        colour_list = ["GFP", "RFP", "Phase"]
        for colour in colour_list:
            if colour in variable_title:
                if colour == "GFP": 
                    return 'green'
                elif colour == 'RFP':
                    return 'red'
                elif colour == 'Phase':
                    return 'cyan'
                else:
                    return "Could not find a suitable colour to plot in, check colour_picker function"

def normalisation(series):
    max_series = np.amax(series)
    norm_factor = 1 / max_series
    scaled_series = series * norm_factor
    return scaled_series

### not sure im using this type of class correctly, but this maps cell_id to normalised_stats path location
class cell_id_stat_location:
    def __init__(self, cell_id, path):
        self.cell_id = cell_id
        self.path = path
        print("Initialised")
cell_info = [cell_id_stat_location("1a", 'C:/Users/Nathan/data/fucci/stats/gold\\cell1a\\normalised_stats.csv'), cell_id_stat_location("1b", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell1b\\normalised_stats.csv'), cell_id_stat_location("2a", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell2a\\normalised_stats.csv'), cell_id_stat_location("2b", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell2b\\normalised_stats.csv'), cell_id_stat_location("3a", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell3a\\normalised_stats.csv'), cell_id_stat_location("3b", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell3b\\normalised_stats.csv'), cell_id_stat_location("4", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell4\\normalised_stats.csv'), cell_id_stat_location("5", 'C:/Users/Nathan/Documents/data/fucci/stats/gold\\cell5\\normalised_stats.csv')] 

Initialised
Initialised
Initialised
Initialised
Initialised
Initialised
Initialised
Initialised


In [ ]:
### Raw data plots 
for cell in cell_ids:
    cell_id = "cell" + cell
    stat_location = os.path.join(cell_id, "stats.csv")

    df = pd.read_csv(os.path.join(root_path, stat_location))
    print(df)
    x = df.index
    x_time = df.index * 4/60 #for plotting the x axis as hours assuming time period between frames is 4mins

    y_variables = df.columns[0:4] #this calls just the first four variables 
    for y in y_variables:
        intensity = df[y]            
        plt.scatter(x_time,[intensity], s= 0.1)
        plt.ylabel(y)
        plt.xlabel('Time (hrs)')
        title = y + ' of cell ' + cell
        plt.title(title)
        plt.show()


In [ ]:
### Normalised data plots

for cell in cell_ids:
    cell_id = "cell" + cell
    stat_location = os.path.join(cell_id, "stats.csv")

    df = pd.read_csv(os.path.join(root_path, stat_location))

    x = df.index
    x_time = df.index * 4/60 #for plotting the x axis as hours assuming time period between frames is 4mins

    y_variables = df.columns[0:4] #this calls just the first four variables 
    for y in y_variables:
        #intensity = df[y] 
        norm_intensity = normalisation(df[y])      
        plt.scatter(x_time,[norm_intensity], s= 0.1)
        #plt.plot(x_time,[norm_intensity], s= 0.1)
        ylabel = "Normalised " + y
        plt.ylabel(ylabel)
        plt.xlabel('Time (hrs)')
        title = y + ' of cell ' + cell
        plt.title(title)
        plt.show()

In [ ]:
### Overlaid normalised plots 
cell_ids = ["1a", "1b"]
for cell in cell_ids:
    cell_id = "cell" + cell
    stat_location = os.path.join(cell_id, "stats.csv")

    df = pd.read_csv(os.path.join(root_path, stat_location))
    norm_df = pd.DataFrame([])

    x = df.index
    x_time = df.index * 4/60 #for plotting the x axis as hours assuming time period between frames is 4mins
    y_variables = df.columns[0:4] #this calls just the first four variables 

    # collect different intensities to plot
    for y in y_variables:
            #intensity = df[y] 
            y_name = 'Normalised ' + y
            norm_intensity = normalisation(df[y]) 
            norm_df[y_name] = norm_intensity   
    
    #plotting intensities on same plot
    for column in norm_df.columns[0:2]: #index in df.columns indicate variables to plot
        colour = colour_picker(column)
        plt.plot(x_time, norm_df[column], linewidth=0.5, color=colour, label = column)
    #plt.plot(x_time, norm_df["Normalised Area"], linewidth=0.5, color="gray", label = "Normalised Area" )
    plt.ylabel("Normalised Intensity")
    plt.xlabel('Time (hrs)')
    plt.legend()
    title = "Normalised intensity of cell " + cell
    #title = norm_df.columns[0]+", "+ norm_df.columns[1]+ ", "+ norm_df.columns[2]+" of cell " + cell
    plt.title(title)
    save_title = title + ".png"
    plt.savefig(os.path.join(root_path, cell_id, save_title), dpi=1000)
    plt.show()       

In [ ]:
# overlaid multicell plots, alignment needed
norm_df = pd.DataFrame([])
norm_df_list = []
cell_ids = ["1a", "1b"]
for cell in cell_ids:
    cell_id = "cell" + cell

    stat_location = os.path.join(cell_id, "stats.csv")

    df = pd.read_csv(os.path.join(root_path, stat_location))    

    x = df.index
    x_time = df.index * 4/60 #for plotting the x axis as hours assuming time period between frames is 4mins
    
    # collect different intensities to plot
    for y in y_variables:
            #intensity = df[y] 
            y_name = 'Normalised ' + y
            norm_intensity = normalisation(df[y]) 
            norm_df[y_name] = norm_intensity
    
    stat_location = os.path.join(cell_id, "normalised_stats.csv")
    norm_stat_file = os.path.join(root_path, stat_location)
    norm_df.to_csv(norm_stat_file, index=False)
    norm_df_list.append(norm_stat_file)
    
#need to edit these dataframes and and align them so that norm_df[i].min is all aligned 
for i, cell in enumerate(cell_ids):
    norm_df = pd.read_csv(cell_info[i].path)
    #now shift index of df so that the three column variables align
    shift1a = 0 
    shift2a = 0
    shift = 0 #amount to shift the df by
    norm_df.shift(periods=shift)

    
# for each channel
for column in norm_df.columns[2:3]: #only showing PHASE atm
    #for each cell
    for i, cell in enumerate(cell_ids):
        #print(cell)
        norm_df = pd.read_csv(cell_info[i].path)
        #colour = colour_picker(column)
        plt.plot(x_time, norm_df[column], linewidth=0.5, label = cell)#, color=colour)

    plt.ylabel("Normalised Intensity")
    plt.xlabel('Time (hrs)')
    title = column+ " of multiple cells"
    plt.title(title)
    plt.legend()
    save_title = title + ".png"
    plt.savefig(os.path.join(root_path, save_title), dpi=1000)
    plt.show()   

In [ ]:
for i, cell in enumerate(cell_ids):

    norm_df = pd.read_csv(cell_info[i].path)

    differences_df = norm_df.diff(periods=50)
    align_point = differences_df["Normalised GFP Intensity"].idxmin(axis=0) ##calculates the largest drop in gfp for alignment
    
    align_dict[cell] = align_point
    index = i+1 
    #print(len(cell_ids))
    #if i+1 > int(cell_ids[-1]):
    if i+1 > len(cell_ids)-1:
        break
    else:
        shift_amount = align_dict[cell_ids[i]] - align_dict[cell_ids[i+1]]
    print(shift_amount)
    #shift = 0 #amount to shift the df by
    #shift_df = norm_df.shift(periods=align_point)

    #plt.scatter(x_time,[shift_df["Normalised GFP Intensity"]], s= 0.1)
    #plt.show()
print(align_dict)

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os

norm_df = pd.read_csv(os.path.join(root_path, stat_location))


#print(norm_df)
def get_data(table,col,title):
    data = norm_df['Normalised GFP Intensity'].astype(float)
    data.columns = {title}
    return data

#%matplotlib notebook
title = 'Normalised GFP Intensity'
d = norm_df
x = np.array(d.index)
y = np.array(d['Normalised GFP Intensity'])

fluorescence = pd.DataFrame(y,x)
print(fluorescence)
#XN,YN = augment(x,y,10)
#augmented = pd.DataFrame(YN,XN)
fluorescence.columns = {title}


Writer = animation.writers['ffmpeg']
writer = Writer(fps=2, metadata=dict(artist='Me'), bitrate=180)

fig = plt.figure(figsize=(10,6))
#plt.xlim(0, 70)
#plt.ylim(0,1)
plt.xlabel('Hours',fontsize=20)
plt.ylabel(title,fontsize=20)
plt.title('Normalised GFP Intensity',fontsize=20)

def animate(i):
    data = norm_df.iloc[:int(i+1)] #select data range
    p = sns.lineplot(x=data.index, y=data[title], data=data, color="r")
    p.tick_params(labelsize=5)
    plt.setp(p.lines,linewidth=2)
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=500, repeat=True)
ani.save('Normalised GFP Intensity.mp4', writer=writer)


NameError: name 'norm_df' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

TWOPI = 2*np.pi

fig, ax = plt.subplots()

t = np.arange(0.0, TWOPI, 0.001)
s = np.sin(t)
l = plt.plot(t, s)

ax = plt.axis([0,TWOPI,-1,1])

redDot, = plt.plot([0], [np.sin(0)], 'ro')

def animate(i):
    redDot.set_data(i, np.sin(i))
    return redDot,

# create animation using the animate() function
myAnimation = animation.FuncAnimation(fig, animate, frames=np.arange(0.0, TWOPI, 0.1), \
                                      interval=10, blit=True, repeat=True)

plt.show()
ani.save('HeroinOverdosesJumpy.mp4', writer=writer)
